In [25]:
!pip install backtesting yfinance --quiet


In [26]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 5)


In [27]:
ticker = "RELIANCE.NS"
print(f"Downloading data for {ticker} ...")

raw = yf.download(
    ticker,
    start="2018-01-01",
    end="2024-01-01",
    auto_adjust=False
)

print("\nRaw columns:")
print(raw.columns)

# Flatten multiindex so backtesting gets standard "OHLCV" format
if isinstance(raw.columns, pd.MultiIndex):
    data = raw.copy()
    data.columns = data.columns.droplevel(1)
else:
    data = raw.copy()

print("\nFinal columns:")
print(data.columns)
print("\nPreview:")
print(data.head())


[*********************100%***********************]  1 of 1 completed


Raw columns:
MultiIndex([('Adj Close', 'RELIANCE.NS'),
            (    'Close', 'RELIANCE.NS'),
            (     'High', 'RELIANCE.NS'),
            (      'Low', 'RELIANCE.NS'),
            (     'Open', 'RELIANCE.NS'),
            (   'Volume', 'RELIANCE.NS')],
           names=['Price', 'Ticker'])

Final columns:
Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

Preview:
Price        Adj Close       Close        High         Low        Open  \
Date                                                                     
2018-01-01  401.864868  415.907104  421.827393  414.878479  421.827393   
2018-01-02  402.483215  416.547119  420.387329  414.375580  417.392883   
2018-01-03  404.095520  418.215790  423.336060  417.415741  422.878876   
2018-01-04  406.525055  420.730194  421.415955  418.627228  419.747284   
2018-01-05  407.828125  422.078827  423.747498  420.707336  421.415955   

Price         Volume  
Date                  
2018-01-01 

In [28]:
class SmaCross(Strategy):
    n_fast = 10
    n_slow = 50

    def init(self):
        price = self.data.Close
        self.sma_fast = self.I(SMA, price, self.n_fast)
        self.sma_slow = self.I(SMA, price, self.n_slow)

    def next(self):
        # Fast SMA crosses above slow -> go long
        if crossover(self.sma_fast, self.sma_slow):
            self.position.close()
            self.buy()

        # Fast SMA crosses below slow -> go short
        elif crossover(self.sma_slow, self.sma_fast):
            self.position.close()
            self.sell()


In [29]:
bt = Backtest(
    data,
    SmaCross,
    cash=100_000,
    commission=0.0005,
    trade_on_close=True
)

stats = bt.run()
stats   #table of performance metrics


Backtest.run:   0%|          | 0/1431 [00:00<?, ?bar/s]

C:\Users\Rugved\AppData\Local\Temp\ipykernel_23896\1597082355.py:9: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


Start                     2018-01-01 00:00:00
End                       2023-12-29 00:00:00
Duration                   2188 days 00:00:00
Exposure Time [%]                    93.58542
Equity Final [$]                  85965.61904
Equity Peak [$]                  140009.52212
Commissions [$]                    3255.46128
Return [%]                          -14.03438
Buy & Hold Return [%]               204.30479
Return (Ann.) [%]                    -2.54031
Volatility (Ann.) [%]                26.18541
CAGR [%]                             -1.72661
Sharpe Ratio                         -0.09701
Sortino Ratio                        -0.13541
Calmar Ratio                         -0.04795
Alpha [%]                            -0.91356
Beta                                 -0.06422
Max. Drawdown [%]                   -52.97331
Avg. Drawdown [%]                   -14.41459
Max. Drawdown Duration     1199 days 00:00:00
Avg. Drawdown Duration      230 days 00:00:00
# Trades                          

In [30]:
bt.plot()


GridPlot(id='p2099', ...)